In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
train = pd.read_csv('../data/train.csv', usecols=['ciudad', 'provincia', 'lat', 'lng'])

In [3]:
train.head(2)

,ciudad,provincia,lat,lng
0,Benito Juárez,Distrito Federal,NaN,NaN
1,La Magdalena Contreras,Distrito Federal,19.310205,-99.227655


# Se verifica si hay coordenadas extrañas.

In [4]:
train.describe()

,lat,lng
count,116512.000000,116512.000000
mean,20.696818,-99.509221
std,3.137884,9.744143
min,-100.886679,-125.859375
25%,19.359846,-100.977908
50%,19.543273,-99.240387
75%,20.740005,-99.134777
max,83.026219,121.036000


##### Hay valores que pueden estar mezclados.
Para México la latitud tiene que ser positiva (Norte) y la longitud negativa (Oeste).

Las coordenadas extremas que enmarcan el territorio mexicano son:

**Norte:** 32° 43´ 06´´ = 32.718333° latitud norte, en la frontera con los Estados Unidos de América.

**Sur:** 14° 32´ 27´´ = 14.540833° latitud norte, frontera con Guatemala.

**Este:** 86° 42´ 36´´ = -86.710000° longitud oeste, en el extremo suroeste de la Isla Mujeres.

**Oeste:** 118° 27´ 24´´ = -118.456667° longitud oeste, en el Océano Pacífico.


In [5]:
train.loc[(train['lat'] < 14) | \
          (train['lat'] > 33) | \
          (train['lng'] > -85) | \
          (train['lng'] < -119)].count()

ciudad       635
provincia    641
lat          641
lng          641
dtype: int64

##### Hay 641 filas con coordenas inválidas. Se rellenan con np.NaN.

In [6]:
indice = train.loc[(train['lat'] < 14) | \
                   (train['lat'] > 33) | \
                   (train['lng'] > -85) | \
                   (train['lng'] < -119)].index

In [7]:
%%time
i = 1
for var in indice:
    clear_output(wait=True)
    train.loc[var, 'lat'] = np.NaN
    train.loc[var, 'lng'] = np.NaN
    print(i, 'de', indice.size)
    i = i + 1

641 de 641
CPU times: user 3.9 s, sys: 311 ms, total: 4.21 s
Wall time: 3.66 s


##### Datos nulos por columna.

In [8]:
train.isnull().sum()

ciudad          372
provincia       155
lat          124129
lng          124129
dtype: int64

##### Datos nulos para laitud y longitud simultáneamente, con "**AND**" .

In [9]:
train.loc[(train['lat'].isnull()) & (train['lng'].isnull())].isnull().sum()

ciudad          212
provincia       118
lat          124129
lng          124129
dtype: int64

##### Datos nulos para laitud y longitud , con "**OR**" inclusivo.

In [10]:
train.loc[(train['lat'].isnull()) | (train['lng'].isnull())].isnull().sum()

ciudad          212
provincia       118
lat          124129
lng          124129
dtype: int64

##### Se ve que coinciden los nulos para ambas coordenadas simultáneamente.

# Se exporta el train sin coordenadas inválidas

In [11]:
train.to_csv('../data/aux_minus_bad_coord_train.csv', index=False)

In [12]:
prueba = pd.read_csv('../data/aux_minus_bad_coord_train.csv')
prueba.head(5)

,ciudad,provincia,lat,lng
0,Benito Juárez,Distrito Federal,NaN,NaN
1,La Magdalena Contreras,Distrito Federal,19.310205,-99.227655
2,Tonalá,Jalisco,NaN,NaN
3,Zinacantepec,Edo. de México,19.301890,-99.688015
4,Zapopan,Jalisco,NaN,NaN


In [13]:
prueba.shape

(240000, 4)